# EDA by oXyut9943

In [1]:
import os

import pandas as pd

In [2]:
df_base_train = pd.read_csv('../../data/inputs/csv_files/train/train_base.csv')
df_base_test = pd.read_csv('../../data/inputs/csv_files/test/test_base.csv')

In [3]:
display(df_base_train)
display(df_base_test)

,case_id,date_decision,MONTH,WEEK_NUM,target
0,0,2019-01-03,201901,0,0
1,1,2019-01-03,201901,0,0
2,2,2019-01-04,201901,0,0
3,3,2019-01-03,201901,0,0
4,4,2019-01-04,201901,0,1
...,...,...,...,...,...
1526654,2703450,2020-10-05,202010,91,0
1526655,2703451,2020-10-05,202010,91,0
1526656,2703452,2020-10-05,202010,91,0
1526657,2703453,2020-10-05,202010,91,0


,case_id,date_decision,MONTH,WEEK_NUM
0,57543,2020-10-06,202010,92
1,57549,2020-10-06,202010,92
2,57551,2020-10-06,202010,92
3,57552,2020-10-07,202010,92
4,57569,2020-10-06,202010,92
5,57630,2020-10-06,202010,92
6,57631,2020-10-06,202010,92
7,57632,2020-10-06,202010,92
8,57633,2020-10-06,202010,92
9,57634,2020-10-06,202010,92


In [4]:
df_static_0_train = pd.read_csv('../../data/inputs/csv_files/train/train_static_0_0.csv')

/tmp/ipykernel_1070/2311328772.py:1: DtypeWarning: Columns (20,45,46,53,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  df_static_0_train = pd.read_csv('../../data/inputs/csv_files/train/train_static_0_0.csv')


In [6]:
df_static_0_train

,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
0,0,NaN,NaN,1917.6,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.000,0.00,NaN,BO,NaN,NaN
1,1,NaN,NaN,3134.0,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.000,0.00,NaN,BO,NaN,NaN
2,2,NaN,NaN,4937.0,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.000,0.00,NaN,BO,AL,NaN
3,3,NaN,NaN,4643.6,0.0000,0.0,1.0,0.0,2.0,0.0,...,1.0,1.0,NaN,NaN,0.000,0.00,NaN,BO,AL,NaN
4,4,NaN,NaN,3390.2,0.0000,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.000,0.00,NaN,BO,AL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003752,2651088,0.0,117624.79,3045.0,4488.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,44848.000,44848.000,44848.000,199664.67,8976.000,FO,NaN,2019-11-12
1003753,2651089,0.0,134887.40,1200.0,4382.8003,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,39392.000,39392.000,39392.000,209840.00,8765.601,FO,NaN,2019-11-12
1003754,2651090,0.0,69186.62,6000.0,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000,0.000,76455.62,NaN,FO,NaN,2019-11-12
1003755,2651091,0.0,117331.00,11565.4,13781.2000,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,55084.402,55084.402,55084.402,168248.31,13781.200,FO,NaN,2019-11-30
